In [18]:
# Data preparation
import numpy as np
import pandas as pd

# Statistics
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import acf, pacf

# Visualization
import plotly.express as px
import plotly.graph_objects as go
import kaleido
from tqdm.notebook import tqdm

# Modeling
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import (
    TimeSeriesSplit, KFold, cross_val_score,
)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import sklearn.tree
import sklearn.ensemble
import sklearn.linear_model

import xgboost as xgb
import lightgbm as lgbm

In [19]:
def perform_adf_test(series: pd.Series = None) -> float:
   '''Perform Augmented Dickey Fuller test

   :param series: time series, defaults to None
   :type series: pd.Series, optional
   :return: pvalue
   :rtype: float
    
   Stationarity means that the statistical properties of a time series, i.e. mean, variance and covariance do not change over time. 
   Many statistical models require the series to be stationary to make effective and precise predictions.
   The null hypothesis of the Augmented Dickey-Fuller is that there is a unit root (series is not stationary).
   The alternative hypothesis is that there is no unit root.'
   '''
   pval = np.round(adfuller(x=series, regression='c', autolag=None)[1], 3)
   return pval

In [20]:
def plot_pacf(series:pd.Series, series_name:str, nlags:int, partial:bool=True, save:bool=False):
    
    '''
    This func estimates the (partial) autocorrelation function of a given time series using statsmodels package and then plots it using plotly package
    Note that the default pacf calculation method is Yule-Walker with sample size-adjustment in denominator for acovf
    Note that the default confidence interval is 95%
    
    :param series: observations of time series for which acf/pacf is calculated
    :type series: pd.Series
    :param series_name: at least the name and the frequency of the time series, without special characters, that will be used to create a title
    :type series_name: str
    :param nlags: number of lags to return autocorrelation for
    :type nlags: int
    :param partial: whether to plot pacf
    :type partial: bool (default True)
    :param save: whether to save the plot to the current workding directory
    :type save: bool (default False)
    '''
    
    if not isinstance(series, pd.Series):
        raise Exception('Time series is not a pd.Series type!')

    # Define the title depending on the bool argument
    title=f'PACF of {series_name}' if partial else f'ACF of {series_name}'
    
    # Calculate the acf/pacf and the confidence intervals
    corr_array, conf_int_array = pacf(series.dropna(), alpha=0.05, nlags=nlags, method='yw') if partial else acf(series.dropna(), alpha=0.05, nlags=nlags)
    
    # Center the confidence intervals so that it's easy to visually inspect if a given correlation is significantly different from zero
    lower_y = conf_int_array[:,0] - corr_array
    upper_y = conf_int_array[:,1] - corr_array
    
    # Create an empty figure
    fig = go.Figure()

    # Plot the correlations using vertical lines
    [fig.add_scatter(x=(x,x), y=(0,corr_array[x]), mode='lines', line_color='#3f3f3f', hoverinfo='skip') 
        for x in np.arange(len(corr_array))]
    
    # Plot the correlations using markers
    # The <extra></extra> part removes the trace name
    fig.add_scatter(
        x=np.arange(len(corr_array)),
        y=corr_array,
        mode='markers',
        marker_color='#1f77b4',
        marker_size=12,
        hovertemplate=
        'Lag %{x}<br>' +
        'Corr: %{y:.2f}<br>' +
        '<extra></extra>'
    )
    
    # Plot the centered confidence intervals
    fig.add_scatter(x=np.arange(len(corr_array)), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)', hoverinfo='skip')
    fig.add_scatter(x=np.arange(len(corr_array)), y=lower_y, mode='lines', fillcolor='rgba(32, 146, 230,0.3)',
        fill='tonexty', line_color='rgba(255,255,255,0)', hoverinfo='skip')
    
    # Prettify the plot
    fig.update_traces(showlegend=False)
    fig.update_xaxes(tickvals=np.arange(start=0, stop=nlags+1))
    fig.update_yaxes(zerolinecolor='#000000')
    fig.update_layout(title=title, title_x=0.5, width=500, height=300, margin=dict(l=0, r=0, b=0, t=30, pad=1))
    # fig.update_layout(title=title, title_x=0.5, width=500, height=300, hovermode=False, margin=dict(l=0, r=0, b=0, t=30, pad=1))

    # Save the plot to the current working directory
    if save:
        fig.write_image(f'''{title.replace(' ', '_')}.png''')

    # Eventually show the plot
    fig.show(engine='kaleido')

In [21]:
def inspect_columns(df):
    
    # Get this function from other's Jupyter
    # A helper function that does a better job than df.info() and df.describe()
    
    result = pd.DataFrame({
        'unique': df.nunique() == len(df),
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum() / len(df)) * 100, 2),
        '1st_row': df.iloc[0],
        'random_row': df.iloc[np.random.randint(low=0, high=len(df))],
        'last_row': df.iloc[-1],
        'dtype': df.dtypes
    })
    return result

In [22]:
import logging

logging.basicConfig(level=logging.INFO)  # Set the logging level
logger = logging.getLogger(__name__)  # Create a logger

def reduce_mem_usage(df: pd.DataFrame=None, verbose: int=0) -> pd.DataFrame:
    
    """
    The function modifies the input DataFrame in place 
    by changing the data types of numeric columns to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    timedelta_cols = df.select_dtypes(include=[np.timedelta64]).columns
    
    # Create a list of valid numeric columns by excluding timedelta columns
    valid_cols = list(set(numeric_cols) - set(timedelta_cols))
    valid_cols = [col for col in valid_cols if col != 'stock_id']

    for col in valid_cols:        
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # For integer columns:
                # Check if the column values fit within the range of a smaller integer type (e.g., int8, int16, int32, int64)
                # Downcast the column to the smallest integer type that can safely store the values
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            # For float columns:
                # Check if the column values fit within the range of a smaller float type (e.g., float16, float32)
                # Downcast the column to the smallest float type that can safely store the values
            else:

                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    
    logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
    end_mem = df.memory_usage().sum() / 1024**2
    logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
    decrease = 100 * (start_mem - end_mem) / start_mem
    logger.info(f"Decreased by {decrease:.2f}%")
    
    # In Kaggle notebooks logger() statements aren't displayed
    print(f"Memory usage of dataframe is {start_mem:.2f} MB")
    print(f"Memory usage after optimization is: {end_mem:.2f} MB")
    print(f"Decreased by {decrease:.2f}%")

    return df

In [23]:
def custom_mae(y_true, y_pred):
    # Calculate the absolute errors
    errors = np.abs(y_true - y_pred)
    
    # Compute the gradient of the MAE
    grad = np.where(y_true > y_pred, -1, 1)
    
    # Hessian is a constant value for MAE
    hess = np.ones_like(y_true)
    
    # Calculate the mean absolute error
    objective = np.mean(errors)
    
    return grad, hess


# ***WAP - Weighted Average Price***


- **Weighted Averaged Price (WAP)**

WAP in a nut shell, it's a price take bid/ask size into consideration. A fair book-based valuation must take two factors into account: the level and the size of orders.

$$ WAP = \frac{BidPrice_{1} * AskSize_{1} + AskPrice_{1} * BidSize_{1}}{BidSize_{1}+AskSize_{1} } $$

As you can see, if two books have both bid and ask offers on the same price level respectively, the one with more offers in place will generate a lower stock valuation, as there are more intended seller in the book. On the other hand, more seller implies a fact of more supply on the market resulting in a lower stock valuation.

The idea behind this weighting is that higher buying pressure tends to drive prices up, while an increase in sellers tends to push prices down.
To illustrate this idea with an extreme example: consider a specific moment where the best bid is 5 with a BidSize of 5000, and the best ask is 6 with an AskSize of 10. In this scenario, the number of buyers is way larger than the number of sellers, suggesting that the price should be very close to 6. The WAP reflects this intuition, as it gives more weight to the ask price.

- **Let's take a look at the following 2 scenarios:**

- **Scenario 1: More Buyer, more bid szie, therefore higher WAP**

        - bid price = 1, ask price =2, with ask size = 1, bid size = 2

$$WAP_{1} = \frac{1 * 1 + 2 * 2}{3}  = \frac{5}{3}$$

- **Scenario 2: More Seller, more ask size,  therefore lower WAP**

        - bid price = 1, ask price =2, with ask size = 2, bid size = 1


$$ WAP_{2} = \frac{1 * 2 + 2 * 1}{3}  = \frac{4}{3}$$


Remark: Note that in most of cases, during the continuous trading hours, an order book should not have the scenario when bid order is higher than the offer, or ask, order. In another word, most likely, the bid and ask should never be in cross.

# ***Target Variable***

- **What is Target**: 

    The 60 second future move in the wap of the stock, less the 60 second future move of the synthetic index. Only provided for the train set.

- **Target Variable Formula**:

$$ Target = (\frac{WAP_{t+60}}{WAP_{t}} - \frac{Index_{t+60}}{Index_{t}})$$

Therefore, I think we can only use the stock information to predict the target


- **Remark**:

    1) The benchmark synthetic index  = The synthetic index is a custom weighted index of Nasdaq-listed stocks constructed by Optiver for this competition.

    2) The unit of the target is basis points, which is a common unit of measurement in financial markets.  A 1 bp price move is equivalent to a 0.01% price move.

# ***Closing Auction Order Book Data***



- **What is closing auction:** 

    - 1) **What is an auction?** Auction is a period of time that lets multiple buyers and sellers trade in a single price and in a single joint transaction. There are many different kinds of auction, for example London Stock Exchange offers **Opening Auction**, **Intra-Day Auction**, and **Closing Auction**. In this competition teams are particularly interested in the type of a **closing auction**.
    
    - 2) Closing auction starts after continuous trading (Market Open session)
    
         - Take London Stock Exchange for example: 
   
            (2-1) Opening Auction Session: 7:50 pm - 8:00 pm  
            (2-2) Intra-Day Auction Session: 12:00 am - 12:02 am  
            (2-3) Closing Auction Session: 16:30 am - 16:35 am  
            (2-4) Rest of the time is regular trading hour or continous trading hour
          
    - 3) **The closing auction** trading period is a trading mechanism with a single price bidding. Buyers and sellers enter buy and sell orders during this period. This period is also known as **call period**. After collecting the relevant orders, the trading system will determine the order that can complete the most transactions based on the interaction of the orders. This event is known as **uncrossing event**.

    - 4) **Why do we care about the closing auction book?** In the last ten minutes of the Nasdaq exchange trading session, market makers like Optiver merge traditional **order book data** with **auction book data(Closing Auction Order Book Data)**. This ability to consolidate information from both sources is critical for providing the best prices to all market participants.
    
    - 5) Bid & ask price of the closing auction book dataset in the call period are now **overlapping**.
    
    - 6) A typical closing auction book data **before** uncrossing event might look like this:


| Bid Size (Buyer)| Price | Ask Size (Seller) |
| --- | --- | --- |
|  | 10 | 1 |
| 3 | 9 | 2 |
| 4 | 8 | 4 |

- **Example of closig auction mechanism:** 

    For the closing auction book example above, we can work through definitions for a few key terms. Suppose the auction uncrossed with the book in this state, then:

    - 1) At a price of 10, 0 lots would be matched since there as no bids >= 10.

    - 2) At a price of 9, 3 lots would be matched, as there are 3 bids >=9 and 6 asks <= 9.

    - 3) At a price of 8, 4 lots would be matched, since are 7 bids>=8, and there are 4 asks<=8.

        - The price which maximises the number of matched lots would be 8, therefore 8 is our uncross price.  
    
    
- **Key Words Terminology with closing auction order book data**

    - 1) **Uncross Price**: The price which maximises the number of matched lots. In the above example will be 8.

    - 2) **Matched Size**: the max number of matched lots by one single price. In the above example will be 4.

    - 3) **Imbalance**: Imbalance refers to the number of unmatched shares. In the above example, the uncross price is 8 and there are 7 bids & 4 asks which can be matched for the price of 8. As a result, we have three unmatched bids remaining, and the **remaining bids** contribute to an imbalance of three lots in the **buy direction**.

- **More Key Words Terminology for Price Explanation:**

    - 1) **Far Price**: The crossing price that will maximize the number of shares matched based on auction interest only. Therfore, Far price refers to the hypothetical uncross price of the auction book, if it were to uncross at the reporting time.

    - 2) **Near Price**: The crossing price that will maximize the number of shares matched **based auction** and **continuous market orders**. Nasdaq provides near price information 5 minutes before the closing cross. Therefore, in the below figure we can see that starting from 300 sec, the near price/far price start to show.

    - 3) **Reference Price**: 

        - If the "Near Price" is between the best bid and ask, then the reference price is equal to the near price

        - If the "Near Price" > best ask, then reference price = best ask

        - If the "Near Price" < best bid, then reference price = best bid So the reference price is the near price bounded between the best bid and ask. **In summary for reference price, it's a uncross price that best represent the price that will maximize the match lots in the order book data.**

# ***Imbalance Buy/Sell Flag***

**- Imbalance** refers to the number of unmatched shares.

| Bid Size (Buyer)| Price | Ask Size (Seller) |
| --- | --- | --- |
|  | 10 | 1 |
| 3 | 9 | 2 |
| 4 | 8 | 4 |


In the above example, the uncross price is 8 and there are 7 bids & 4 asks which can be matched, therefore we are left with 3 bids unmatched. the left over are bid size, therefore, there is an imbalance of 3 lots in the buy direction.

- Imbalance_size: The amount unmatched at the current reference price (in USD).

- Imbalance_buy_sell_flag: An indicator reflecting the direction of auction imbalance.

        - buy-side imbalance: 1
        - sell-side imbalance: -1
        - no imbalance: 0
        
        
 - Remark: during my EDA process, I found that "imbalance_buy_sell_flag * imbalance_size" better reflect the information of bid/ask size.

In [24]:
df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

In [25]:
inspect_columns(df)

,unique,cardinality,with_null,null_pct,1st_row,random_row,last_row,dtype
stock_id,False,200,False,0.00,0,172,199,int64
date_id,False,481,False,0.00,0,249,480,int64
seconds_in_bucket,False,55,False,0.00,0,20,540,int64
imbalance_size,False,2971863,True,0.00,3180602.69,620748.4,1884285.71,float64
imbalance_buy_sell_flag,False,3,False,0.00,1,1,-1,int64
reference_price,False,28741,True,0.00,0.999812,1.000161,1.002129,float64
matched_size,False,2948862,True,0.00,13380276.64,2458019.33,24073677.32,float64
far_price,False,95739,True,55.26,NaN,NaN,1.000859,float64
near_price,False,84625,True,54.55,NaN,NaN,1.001494,float64
bid_price,False,28313,True,0.00,0.999812,0.999313,1.002129,float64


In [26]:
# Seconds_in_bucket ends at 540, which makes perfect sense
px.line(df['seconds_in_bucket'].unique(), markers='.')

In [27]:
# Numbers of stocks per date in time has benn increasing
px.line(df.groupby(by=df['date_id'])['stock_id'].nunique(), markers='.')

In [28]:
fig = px.line(df.groupby(by=df['date_id'])['stock_id'].nunique(), markers='.')
fig.show(engine='kaleido')

In [29]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
# Take a look at what a single session might like
df.loc[(df['stock_id'] == 0) & (df['date_id'] == 0)]

In [ ]:
adf_pvalues = np.empty(df['stock_id'].nunique())
for i, stock in tqdm(enumerate(df['stock_id'].unique())):
    adf_pvalues[i] = perform_adf_test(df.loc[df['stock_id'] == stock, 'target'].dropna())

In [ ]:
# Target is stationary for each of the stock
adf_pvalues[adf_pvalues > 0]

In [ ]:
# Target is highly correlated with itself for the same stock
stock = 0
plot_pacf(series=df.loc[df['stock_id'] == stock, 'target'].dropna(), series_name=f'stock_id = {stock}', nlags=50, partial=False)
plot_pacf(series=df.loc[df['stock_id'] == stock, 'target'].dropna(), series_name=f'stock_id = {stock}', nlags=50)

In [ ]:
# Target is highly correlated with itself for the same stock
stock = 100
plot_pacf(series=df.loc[df['stock_id'] == stock, 'target'].dropna(), series_name=f'stock_id = {stock}', nlags=50, partial=False)
plot_pacf(series=df.loc[df['stock_id'] == stock, 'target'].dropna(), series_name=f'stock_id = {stock}', nlags=50)

In [ ]:
# Target exhibits some autocorrelation that I could take advantage of
# Since target is wap(t+60) / wap, I could use target.shift(7) as a feature
# Nevertheless, the model should be stock-agnostic, since the organizers highlighted
# that the number of different stocks may change in time
# So probably they will include a couple of new stocks to the test set 

In [ ]:
df.loc[(df['stock_id'] == 0) & (df['date_id'] == 7)].corr(method='spearman').sort_values(by=['target']).round(2)[['target']]

In [ ]:
px.line(df.loc[(df['seconds_in_bucket'] == 0), 'target'])

In [ ]:
# There is no autocorrelation for a single time period, so that I can iterate over 'date', `each seconds_in_bucket` values
# I don't need to be aware of data leakage
plot_pacf(series=df.loc[df['seconds_in_bucket'] == 0, 'target'].dropna(), series_name=f'seconds_in_bucket = {0}', nlags=100, partial=False)
plot_pacf(series=df.loc[df['seconds_in_bucket'] == 0, 'target'].dropna(), series_name=f'seconds_in_bucket = {0}', nlags=100)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df['imbalance_size_vs_matched_size'] = df['imbalance_size'] / df['matched_size']
df['imbalance_size_vs_matched_size*imbalance_buy_sell_flag'] = df['imbalance_size_vs_matched_size'] * df['imbalance_buy_sell_flag']
df['bid_ask_spread_relative'] = (df['bid_price'] - df['ask_price']) / df['ask_price']

df[['far_price', 'near_price']] = df[['far_price', 'near_price']].fillna(0)
df['near_price_vs_reference_price'] = df['near_price'] / df['reference_price']
df['far_price_vs_reference_price'] = df['far_price'] / df['reference_price']
df['bid_price_vs_reference_price'] = df['bid_price'] / df['reference_price']
df['ask_price_vs_reference_price'] = df['ask_price'] / df['reference_price']

In [ ]:
# Define features
features = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'near_price_vs_reference_price', 'far_price_vs_reference_price',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

In [ ]:
df[features].isna().sum()

In [ ]:
df.loc[df['ask_price_vs_reference_price'].isna()].isna().sum()

In [ ]:
# These are certainly uninformative rows, drop them without any second thoughts
df = df.loc[~df['ask_price_vs_reference_price'].isna()].copy()

In [ ]:
df = reduce_mem_usage(df)

In [ ]:
# IDEAS
# Attempt 1: build two different models for 0-290 seconds and 300-540 (after near_price is being published) AND don't care about the time dimension
# Attempt 2: build two different models for 0-60 seconds and 70-540 (after you can obtain target shift(7) without data leakage) AND and do care about the time dimension (remember to iterate over stocks)

In [ ]:
agg_list = ['sum', 'min', 'max', 'mean', 'median', lambda x: x.mean()/x.std(), pd.Series.kurtosis, pd.Series.skew, 'count']

In [ ]:
px.histogram(df['target'], width=500, height=500)

In [ ]:
df[['target']].agg(func=agg_list).round(2)

In [ ]:
df[['target']].describe().round(2)

In [ ]:
df['target'].quantile(q=0.95)

In [ ]:
df['target'].quantile(q=0.05)

In [ ]:
df.loc[df['target'] < df['target'].quantile(q=0.05), 'date_id'].nunique()

In [ ]:
tscv = TimeSeriesSplit(n_splits=10)
kfcv = KFold(n_splits=10)

In [ ]:
# Define features for the Model I
features_1 = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

# Define features for the Model II
features_2 = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'near_price_vs_reference_price', 'far_price_vs_reference_price',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

In [ ]:
X_1 = df.loc[df['seconds_in_bucket'] < 300, features_1].copy()
y_1 = df.loc[df['seconds_in_bucket'] < 300, 'target'].copy()

X_2 = df.loc[df['seconds_in_bucket'] >= 300, features_1].copy()
y_2 = df.loc[df['seconds_in_bucket'] >= 300, 'target'].copy()

In [ ]:
from sklearn.metrics import get_scorer_names

In [ ]:
get_scorer_names()

In [ ]:
model = LinearRegression(fit_intercept=True)

# Compute 10-fold TimeSeriesSplit CV scores
cv_scores = np.round(-cross_val_score(estimator=model, X=X_1, y=y_1, cv=tscv, scoring='neg_mean_absolute_error'), 2)
print(f'tscv: {cv_scores} | {np.round(np.mean(cv_scores), 2)}')

# Compute 10-fold KFold CV scores
cv_scores = np.round(-cross_val_score(estimator=model, X=X_1, y=y_1, cv=kfcv, scoring='neg_mean_absolute_error'), 2)
print(f'kfcv: {cv_scores} | {np.round(np.mean(cv_scores), 2)}')

In [ ]:
model = LinearRegression()

# Compute 10-fold TimeSeriesSplit CV scores
cv_scores = np.round(-cross_val_score(estimator=model, X=X_2, y=y_2, cv=tscv, scoring='neg_mean_absolute_error'), 2)
print(f'tscv: {cv_scores} | {np.round(np.mean(cv_scores), 2)}')

# Compute 10-fold KFold CV scores
cv_scores = np.round(-cross_val_score(estimator=model, X=X_2, y=y_2, cv=kfcv, scoring='neg_mean_absolute_error'), 2)
print(f'kfcv: {cv_scores} | {np.round(np.mean(cv_scores), 2)}')

### Attempt 1: 
* Model I for 0-290 seconds
* Model II for 300-540 (after near_price is being published)

In [ ]:
def get_kfold_cv_scores_separate(
    list_models: List = None,
    data: pd.DataFrame = None,
    features_1: List[str] = None,
    features_2: List[str] = None,
    n_splits: int = 10,
) -> Dict:
    
    # Use dates to make the splits, instead of using rows directly
    dates = data['date_id'].unique()
    kfcv = KFold(n_splits=n_splits)
    PURGE = 1
 
    # Store performance in a dict
    stats_CV = {}
    for train, test in tqdm(kfcv.split(dates)):
        
        # Purge
        if test.min() > train.max():
            train = train[:-PURGE]
        elif test.max() < train.min():
            train = train[PURGE:]
        elif test.min() > train.min() and test.max() < train.max():
            train = np.concatenate((train[train < test.min()][:-PURGE], train[train > test.max()][PURGE:]))
        
        fold_train = data.loc[data['date_id'].isin(train), :]
        fold_test = data.loc[data['date_id'].isin(test), :]
        
        for model in list_models:
            
            model_name = model.__class__.__name__
            
            if model_name == 'Lasso':
                
                # Standardize features_1 using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1])
                
                # Fit model I and make predictions
                model.fit(
                    X=scaler.transform(fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1]),
                    y=fold_train.loc[fold_train['seconds_in_bucket'] < 300, 'target']
                )
                preds_1 = model.predict(scaler.transform(fold_test.loc[fold_test['seconds_in_bucket'] < 300, features_1]))
                
                # Standardize features_2 using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2])
                
                # Fit model II and make predictions
                model.fit(
                    X=scaler.transform(fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2]),
                    y=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, 'target']
                )
                preds_2 = model.predict(scaler.transform(fold_test.loc[fold_test['seconds_in_bucket'] >= 300, features_2]))
            
            else:
                
                # Just fit two models and make predictions
                model.fit(
                    X=fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1],
                    y=fold_train.loc[fold_train['seconds_in_bucket'] < 300, 'target']
                )
                preds_1 = model.predict(fold_test.loc[fold_test['seconds_in_bucket'] < 300, features_1])
            
                model.fit(
                    X=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2],
                    y=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, 'target']
                )
                preds_2 = model.predict(fold_test.loc[fold_test['seconds_in_bucket'] >= 300, features_2])
            
            # Combine predictions and targets to compute the metric
            preds_concat = np.concatenate([preds_1, preds_2])
            fold_test_y_concat = pd.concat(objs=[
                fold_test.loc[fold_test['seconds_in_bucket'] < 300, 'target'],
                fold_test.loc[fold_test['seconds_in_bucket'] >= 300, 'target']
            ])
            score = mean_absolute_error(preds_concat, fold_test_y_concat)          
            
            if model_name in stats_CV:
                stats_CV[model_name].append(score)
            else:
                stats_CV[model_name] = [score]
                
    return stats_CV

In [ ]:
def get_walk_forward_fixed_origin_cv_scores_separate(
    list_models: List = None,
    data: pd.DataFrame = None,
    features_1: List[str] = None,
    features_2: List[str] = None,
    n_splits: int = 10,
) -> Dict:
    
    # Use dates to make the splits, instead of using rows directly
    dates = data['date_id'].unique()
    tscv = TimeSeriesSplit(n_splits=n_splits)
    PURGE = 1
 
    # Store performance in a dict
    stats_CV = {}
    for train, test in tqdm(tscv.split(dates)):
        
        # Purge 1 date
        train = train[:-PURGE]
        
        fold_train = data.loc[data['date_id'].isin(train), :]
        fold_test = data.loc[data['date_id'].isin(test), :]
        
        for model in list_models:
            
            model_name = model.__class__.__name__
            
            if model_name == 'Lasso':
                
                # Standardize features_1 using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1])
                
                # Fit model I and make predictions
                model.fit(
                    X=scaler.transform(fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1]),
                    y=fold_train.loc[fold_train['seconds_in_bucket'] < 300, 'target']
                )
                preds_1 = model.predict(scaler.transform(fold_test.loc[fold_test['seconds_in_bucket'] < 300, features_1]))
                
                # Standardize features_2 using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2])
                
                # Fit model II and make predictions
                model.fit(
                    X=scaler.transform(fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2]),
                    y=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, 'target']
                )
                preds_2 = model.predict(scaler.transform(fold_test.loc[fold_test['seconds_in_bucket'] >= 300, features_2]))
            
            else:
                
                # Just fit two models and make predictions
                model.fit(
                    X=fold_train.loc[fold_train['seconds_in_bucket'] < 300, features_1],
                    y=fold_train.loc[fold_train['seconds_in_bucket'] < 300, 'target']
                )
                preds_1 = model.predict(fold_test.loc[fold_test['seconds_in_bucket'] < 300, features_1])
            
                model.fit(
                    X=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, features_2],
                    y=fold_train.loc[fold_train['seconds_in_bucket'] >= 300, 'target']
                )
                preds_2 = model.predict(fold_test.loc[fold_test['seconds_in_bucket'] >= 300, features_2])
            
            # Combine predictions and targets to compute the metric
            preds_concat = np.concatenate([preds_1, preds_2])
            fold_test_y_concat = pd.concat(objs=[
                fold_test.loc[fold_test['seconds_in_bucket'] < 300, 'target'],
                fold_test.loc[fold_test['seconds_in_bucket'] >= 300, 'target']
            ])
            score = mean_absolute_error(preds_concat, fold_test_y_concat)          
            
            if model_name in stats_CV:
                stats_CV[model_name].append(score)
            else:
                stats_CV[model_name] = [score]
                
    return stats_CV

In [ ]:
# Test if everything is okay
statsCV = get_kfold_cv_scores_separate(list_models=[LinearRegression(fit_intercept=True)], data=df, features_1=features_1, features_2=features_2, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
# Test if everything is okay
statsCV = get_walk_forward_fixed_origin_cv_scores_separate(list_models=[LinearRegression(fit_intercept=True)], data=df, features_1=features_1, features_2=features_2, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
list_models = [
    sklearn.linear_model.LinearRegression(fit_intercept=True), # probably the simplest model possible for this problem 
    sklearn.linear_model.RANSACRegressor( # https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html#sphx-glr-auto-examples-linear-model-plot-ransac-py
        estimator=None, # if estimator is None, then LinearRegression is used
        loss='absolute_error',
        random_state=42,
    ),
    sklearn.linear_model.Lasso(
        alpha=1, # alpha is a constant that multiplies the L1 term, controlling regularization strength. I require maximum penalty.
        fit_intercept=False, # if set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
    ),
    xgb.XGBRegressor( # pre pruning
        objective='reg:absoluteerror', # regression with L1 error
        seed=42,
        max_depth=5,
        learning_rate=0.3,
        min_child_weight=100, # minimum sum of instance weight (hessian) needed in a child
        gamma=1, # minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be. range: [0,∞]
        n_jobs=-1,
    ), 
    lgbm.LGBMRegressor( # pre pruning
        objective=custom_mae,
        random_state=42,
        max_depth=5,
        learning_rate=0.3,
        min_child_weight=100, # minimum sum of instance weight (hessian) needed in a child
        min_child_samples=50, # minimum number of data points required in a leaf, it helps to control the minimum size of leaves
        # max_leaves=31, maximum tree leaves for base learners is 31, so I commented this line to avoid lgbm warnings 
        n_jobs=-1,
    ),
]

In [ ]:
statsCV = get_kfold_cv_scores_separate(list_models=list_models, data=df, features_1=features_1, features_2=features_2, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
statsCV = get_walk_forward_fixed_origin_cv_scores_separate(list_models=list_models, data=df, features_1=features_1, features_2=features_2, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

### Attempt 2: 
* One model for 0-540 seconds

In [ ]:
def get_kfold_cv_scores_combined(
    list_models: List = None,
    data: pd.DataFrame = None,
    features: List[str] = None,
    n_splits: int = 10,
) -> Dict:
    
    # Use dates to make the splits, instead of using rows directly
    dates = data['date_id'].unique()
    kfcv = KFold(n_splits=n_splits)
    PURGE = 1
 
    # Store performance in a dict
    stats_CV = {}
    for train, test in tqdm(kfcv.split(dates)):
        
        # Purge
        if test.min() > train.max():
            train = train[:-PURGE]
        elif test.max() < train.min():
            train = train[PURGE:]
        elif test.min() > train.min() and test.max() < train.max():
            train = np.concatenate((train[train < test.min()][:-PURGE], train[train > test.max()][PURGE:]))
        
        fold_train = data.loc[data['date_id'].isin(train), :]
        fold_test = data.loc[data['date_id'].isin(test), :]
        
        for model in list_models:
            
            model_name = model.__class__.__name__
            
            if model_name == 'Lasso':
                
                # Standardize features using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train[features])
                
                # Fit the model and make predictions
                model.fit(
                    X=scaler.transform(fold_train[features]),
                    y=fold_train['target']
                )
                preds = model.predict(scaler.transform(fold_test[features]))
            
            else:
                
                # Just fit the model and make predictions
                model.fit(
                    X=fold_train[features],
                    y=fold_train['target']
                )
                preds = model.predict(fold_test[features])
                
            # Calculate score
            score = mean_absolute_error(preds, fold_test['target'])          
            
            if model_name in stats_CV:
                stats_CV[model_name].append(score)
            else:
                stats_CV[model_name] = [score]
                
    return stats_CV

In [ ]:
def get_walk_forward_fixed_origin_cv_scores_combined(
    list_models: List = None,
    data: pd.DataFrame = None,
    features: List[str] = None,
    n_splits: int = 10,
) -> Dict:
    
    # Use dates to make the splits, instead of using rows directly
    dates = data['date_id'].unique()
    tscv = TimeSeriesSplit(n_splits=n_splits)
    PURGE = 1
 
    # Store performance in a dict
    stats_CV = {}
    for train, test in tqdm(tscv.split(dates)):
        
        # Purge 1 date
        train = train[:-PURGE]
        
        fold_train = data.loc[data['date_id'].isin(train), :]
        fold_test = data.loc[data['date_id'].isin(test), :]
        
        for model in list_models:
            
            model_name = model.__class__.__name__
            
            if model_name == 'Lasso':
                
                # Standardize features using only the training set (avoid the information leakage)
                scaler = StandardScaler()
                scaler.fit(fold_train[features])
                
                # Fit the model and make predictions
                model.fit(
                    X=scaler.transform(fold_train[features]),
                    y=fold_train['target']
                )
                preds = model.predict(scaler.transform(fold_test[features]))
            
            else:
                
                # Just fit the model and make predictions
                model.fit(
                    X=fold_train[features],
                    y=fold_train['target']
                )
                preds = model.predict(fold_test[features])
                
            # Calculate score
            score = mean_absolute_error(preds, fold_test['target'])          
            
            if model_name in stats_CV:
                stats_CV[model_name].append(score)
            else:
                stats_CV[model_name] = [score]
                
    return stats_CV

In [ ]:
# Define features
features = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'near_price_vs_reference_price', 'far_price_vs_reference_price',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

In [ ]:
statsCV = get_kfold_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
statsCV = get_walk_forward_fixed_origin_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

### Attempt 3: 
* One model for 0-540 seconds (default models' params)

In [ ]:
list_models = [
    sklearn.linear_model.LinearRegression(fit_intercept=True), # probably the simplest model possible for this problem
    sklearn.linear_model.Lasso(
        alpha=1, # alpha is a constant that multiplies the L1 term, controlling regularization strength. I require maximum penalty.
        fit_intercept=False, # If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
    ),
    xgb.XGBRegressor(
        objective='reg:absoluteerror',
        seed=42,
        n_jobs=-1,
    ), 
    lgbm.LGBMRegressor(
        objective=custom_mae,
        random_state=42,
        n_jobs=-1,
    ),
]

In [ ]:
# Define features
features = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'near_price_vs_reference_price', 'far_price_vs_reference_price',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

In [ ]:
statsCV = get_kfold_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
statsCV = get_walk_forward_fixed_origin_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

### Attempt 4: 
* One model for 0-540 seconds (default models' params) AND objective function is not MAE this time

In [ ]:
list_models = [
    sklearn.linear_model.LinearRegression(fit_intercept=True), # probably the simplest model possible for this problem
    sklearn.linear_model.Lasso(
        alpha=1, # alpha is a constant that multiplies the L1 term, controlling regularization strength. I require maximum penalty.
        fit_intercept=False, # If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
    ),
    xgb.XGBRegressor(
        seed=42,
        n_jobs=-1,
    ), 
    lgbm.LGBMRegressor(
        random_state=42,
        n_jobs=-1,
    ),
]

In [ ]:
# Define features
features = [
    'imbalance_size_vs_matched_size', 'imbalance_size_vs_matched_size*imbalance_buy_sell_flag', 'bid_ask_spread_relative',
    'near_price_vs_reference_price', 'far_price_vs_reference_price',
    'bid_price_vs_reference_price', 'ask_price_vs_reference_price',
]

In [ ]:
statsCV = get_kfold_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

In [ ]:
statsCV = get_walk_forward_fixed_origin_cv_scores_combined(list_models=list_models, data=df, features=features, n_splits=10)
for key, item in statsCV.items():
    print(f'{key} model obtained an average score of {str(np.round(np.mean(item), 3))} in this CV scheme, with a standard deviation of {str(np.round(np.std(item), 3))}')
fig = px.line(data_frame=statsCV, markers='.')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=30, pad=1))
fig.update_xaxes(tickvals=np.arange(start=0, stop=10))
fig.show()

# Key takeaways:
    * Near price is remarkably important (models for 0-290 seconds have MAE of ~7, whereas models for 300-540 seconds have MAE of ~below 6) regardless of kind of cross-validation scheme.
    * There's no significant difference between having two separate models for different periods of time and having a single model.
    * RANSAC model is terribly bad.
    * Other models perform amazingly similar to each other, even without prepruning. This push me to select the OLS model for final submission.
    * There's no significant difference between average scores of KFold CV and Walk Forward Time Series CV, thus relearning is not necessary.
    * In sum, I will train the OLS model with full train data and won't relearn it with new test data during the submission phase.